<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Importar-Librerías" data-toc-modified-id="Importar-Librerías-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Importar Librerías</a></span></li><li><span><a href="#Preprocesamiento-de-Datos" data-toc-modified-id="Preprocesamiento-de-Datos-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Preprocesamiento de Datos</a></span><ul class="toc-item"><li><span><a href="#Construcción-del-Dataset-de-Productos" data-toc-modified-id="Construcción-del-Dataset-de-Productos-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Construcción del Dataset de Productos</a></span><ul class="toc-item"><li><span><a href="#Lectura-de-los-Datos" data-toc-modified-id="Lectura-de-los-Datos-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Lectura de los Datos</a></span></li><li><span><a href="#Transformación-de-los-Datos" data-toc-modified-id="Transformación-de-los-Datos-2.1.2"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>Transformación de los Datos</a></span></li></ul></li></ul></li><li><span><a href="#Caracterización-de-los-Descuentos:-General" data-toc-modified-id="Caracterización-de-los-Descuentos:-General-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Caracterización de los Descuentos: General</a></span></li><li><span><a href="#Caracterización-de-los-Descuentos:-Celulares-y-Teléfonos" data-toc-modified-id="Caracterización-de-los-Descuentos:-Celulares-y-Teléfonos-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Caracterización de los Descuentos: Celulares y Teléfonos</a></span></li></ul></div>

# Pulling Data - Mercado Libre API

El propósito de este notebook es realizar el primer paso del reto de DS y ML propuesto por el equipo Meli: Obtener los datos de ciertos productos a través de la [API](https://api.mercadolibre.com/) de Mercado Libre.

En este notebook se establece la conexión con la API y se obtiene através del buscador de Mercado Libre ciertos productos para construir un dataset que será usuado tanto para la parte exploratoria como la del modelo de ML.

## Importar Librerías

In [1]:
# Cambia el ancho de las celdas
from IPython.display import HTML, display
display(HTML(data="""<style>div#notebook-container{width: 60%;}div#menubar-container{width: 65%;} div#maintoolbar-container{width: 99%;}</style>"""))

In [2]:
# Procesamiento de datos
import os
import re
import pandas as pd
import numpy as np
import datetime as dt
import itertools as it

# Visualización
import matplotlib.pyplot as plt
import matplotlib.font_manager
from matplotlib.ticker import PercentFormatter
import chart_studio.plotly as py
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots

## Preprocesamiento de Datos

En esta sección se realiza un proceso general de limpieza y organización de los datos previo a la exploración de los datos.

### Construcción del Dataset de Productos

#### Lectura de los Datos
 
En esta subsección se leen el dataset de productos, se imprimen sus dimensiones, sus columnas y se mencionan sus tipos de variables.

In [3]:
# Se lee el .csv con el datset construido
products = pd.read_csv('../Data/Datasets/PRODUCTOS.csv')
products.columns = products.columns.str.upper() # Pone los nombres de las columnas en mayusculas
print('LAS DIMENSIONES DE LA BASE DE PRODUCTOS SON: {}'.format(products.shape))

LAS DIMENSIONES DE LA BASE DE PRODUCTOS SON: (14574, 50)


In [4]:
# Printea la columnas del dataset
products.columns

Index(['ITEM_ID', 'ITEM_SITE_ID', 'ITEM_TITLE', 'ITEM_PRICE',
       'ITEM_SALE_PRICE', 'ITEM_CURRENCY_ID', 'ITEM_AVAILABLE_QUANTITY',
       'ITEM_SOLD_QUANTITY', 'ITEM_BUYING_MODE', 'ITEM_LISTING_TYPE_ID',
       'ITEM_STOP_TIME', 'ITEM_CONDITION', 'ITEM_PERMALINK', 'ITEM_THUMBNAIL',
       'ITEM_THUMBNAIL_ID', 'ITEM_ACCEPTS_MERCADOPAGO', 'ITEM_ORIGINAL_PRICE',
       'ITEM_CATEGORY_ID', 'ITEM_OFFICIAL_STORE_ID', 'ITEM_DOMAIN_ID',
       'ITEM_CATALOG_PRODUCT_ID', 'ITEM_ORDER_BACKEND',
       'ITEM_USE_THUMBNAIL_ID', 'SEARCH_CATEGORY_ID', 'SEARCH_CATEGORY_NAME',
       'SEARCH_OFFSET', 'SELLER_ID', 'SELLER_REP_TRANSACTIONS_TOTAL',
       'SELLER_REP_TRANSACTIONS_CANCELED', 'SELLER_REP_RATING_NEG',
       'SELLER_REP_RATING_POS', 'SELLER_REP_RATING_NEU',
       'SELLER_TRANSACTIONS_COMPLETED', 'SELLER_STATUS',
       'SELLER_METRICS_CLAIMS_RATE', 'SELLER_METRICS_CLAIMS_VALUE',
       'SELLER_METRICS_CLAIMS_PERIOD', 'SELLER_METRICS_DELAY_RATE',
       'SELLER_METRICS_DELAY_VALUE', 'SEL

Se puede ver claramente que se encuentran los siguientes tipos de variables:
    
1. Tipo `ITEM`: Variables asociadas al ítem.


2. Tipo `SELLER`: Variables asociadas al vendedor.


3. Tipo `SHIPPING`: Variables asociadas al envío.


4. Tipo `ADRESS`: Variable que indica el departamento donde se encuentra el ítem.


5. Tipo `SEARCH`: Variables asociadas a la búsqueda: categoría y offset.

#### Transformación de los Datos

En esta subsección se arreglan, limpian y transforman el dataset de productos.

In [5]:
# Arreglo a la ADRESS_STATE_ID: para algunos productos el ADRESS_STATE_ID hay que arreglar su dato
# Se uso el ITEM_PERMALINK para entender cual era el verdadero dato de ADRESS_STATE_ID
cambios_de_direccion = {
                        'TUNPUEJPR1gxMDljZA': 'CO-DC', 
                        'TUNPUEFOVGFiZWI3': 'CO-ANT ',
                        'TUNPUENVTmE3NmQ4': 'CO-CUN',
                        'TUNPUFZBTGExNmNjNg': 'CO-VAC',
                        'TUNPUFJJU2ExMWIyYg': 'CO-RIS',
                        'TUNPUEFUTG9mNDk5': 'CO-ATL',
                        'TUNPUFRPTGExNGZkNA': 'CO-TOL',
                        'TUNPUFNBTnJlMjMw': 'CO-SAN',
                        'TUNPUEJPTHI1Mzlk': 'CO-BOL',
                        'TUNPUE1BR2FiZjQ0': 'CO-MAG',
                        'TUNPUENBTHNjODY4': 'CO-CAL',
                        'TUNPUE1FVGExNzFjNQ': 'CO-MET',
                        'TUNPUFFVSW9kYmZm': 'CO-QUI',
                        'TUNPUE5PUnIxNDkyZg': 'CO-NSA', 
                        'TUNPUEFSQ2E4Zjc3': 'CO-SAP',
                        'TUNPUEJPWWE4YzMz': 'CO-BOY',
                        'TUNPUENBVWExM2Q1NQ': 'CO-CAU',
                        'TUNPUENPUmFkZGIw': 'CO-COR',
                        'TUNPUEdVQWExOTYx': 'CO-LAG',
                        'TUNPUE5BUm8xYzk4': 'CO-NAR', 
                        'TUNPUFNVQ2U4ZWQ0': 'CO-SUC',
                        'TUNPUENBU2U2OWIy': 'CO-CAS',
                        'TUNPUENFU3IxODA4Mg': 'CO-CES',
                        'TUNPUEFNQXMxMzQ2YQ': 'CO-AMA'
                       }

# Arregla un error que se detecto en las direcciones
products.loc[products['ADRESS_STATE_ID'].isin(cambios_de_direccion.keys()), 'ADRESS_STATE_ID'] = \
products.loc[products['ADRESS_STATE_ID'].isin(cambios_de_direccion.keys()), 'ADRESS_STATE_ID'].map(cambios_de_direccion)

In [6]:
## Ajuste a la condicion del item

# Se cambia nan por not_specified
products.loc[products['ITEM_CONDITION'].isna(), 'ITEM_CONDITION'] = 'not_specified'

# Ajustes al precio 

# Se quitan productos que no tienen precio (generalmente precio a convenir)
# Es decir este analisis aplica para productos con precio fijo
products = products.loc[~products['ITEM_PRICE'].isna()]
products.reset_index(drop=True, inplace=True)

# Se encontro en google la tasa de cambio de dolar a cop el dia 7 de marzo (cuando se descargaron los datos)
exchange_rate_usd_to_cop = 3142.99 
products.loc[products['ITEM_CURRENCY_ID'] == 'USD', 'ITEM_PRICE'] *= exchange_rate_usd_to_cop
products.loc[products['ITEM_CURRENCY_ID'] == 'USD', 'ITEM_CURRENCY_ID'] = 'COP'

# Cambia ITEM_ORIGINAL_PRICE por el ITEM_PRICE como se indica en el documento
products.loc[products['ITEM_ORIGINAL_PRICE'].isna(), 'ITEM_ORIGINAL_PRICE'] = \
products.loc[products['ITEM_ORIGINAL_PRICE'].isna(), 'ITEM_PRICE']

# Crea las variables de descuento absoluto y la tasa
products['ITEM_DISCOUNT_VALUE'] = products['ITEM_ORIGINAL_PRICE'] - products['ITEM_PRICE']
products['ITEM_DISCOUNT_RATE'] = products['ITEM_DISCOUNT_VALUE']/products['ITEM_ORIGINAL_PRICE']

# Crea la variable de venta total: This is an approximation
products['ITEM_SELL_TOTAL'] = products['ITEM_PRICE']*products['ITEM_SOLD_QUANTITY']

In [7]:
## Ajuste a variables del vendedor

# Cambio de nan to not_specified para SELLER_STATUS
products.loc[products['SELLER_STATUS'].isna(), 'SELLER_STATUS'] = 'not_specified'
products.loc[products['SELLER_LEVEL_ID'].isna(), 'SELLER_LEVEL_ID'] = 'not_specified'

# Cambio nan to 0 para SELLER_METRICS_SALES_PERIOD
products.loc[products['SELLER_METRICS_SALES_PERIOD'].isna(), 'SELLER_METRICS_SALES_COMPLETED'] = 0
products.loc[products['SELLER_METRICS_SALES_PERIOD'].isna(), 'SELLER_METRICS_SALES_PERIOD'] = '60 days'
products.loc[products['SELLER_METRICS_SALES_PERIOD'] == '60 months', 'SELLER_METRICS_SALES_COMPLETED'] *= (60/(30*60))
products.loc[products['SELLER_METRICS_SALES_PERIOD'] == '3 months', 'SELLER_METRICS_SALES_COMPLETED'] *= (60/90)
products.loc[products['SELLER_METRICS_SALES_PERIOD'] == '365 days', 'SELLER_METRICS_SALES_COMPLETED'] *= (60/365)
products['SELLER_METRICS_SALES_PERIOD'] = '60 days'

# Estandarizacion de las metricas absolutas del vendedor (por periodo de tiempo)

# Cancelaciones 
products.loc[products['SELLER_METRICS_CANCELLATIONS_PERIOD'] == '60 months', 'SELLER_METRICS_CANCELLATIONS_VALUE'] *= (60/(30*60))
products.loc[products['SELLER_METRICS_CANCELLATIONS_PERIOD'] == '3 months', 'SELLER_METRICS_CANCELLATIONS_VALUE'] *= (60/90)
products.loc[products['SELLER_METRICS_CANCELLATIONS_PERIOD'] == '365 days', 'SELLER_METRICS_CANCELLATIONS_VALUE'] *= (60/365)
products['SELLER_METRICS_CANCELLATIONS_PERIOD'] = '60 days'

# Quejas 
products.loc[products['SELLER_METRICS_CLAIMS_PERIOD'] == '60 months', 'SELLER_METRICS_CLAIMS_VALUE'] *= (60/(30*60))
products.loc[products['SELLER_METRICS_CLAIMS_PERIOD'] == '3 months', 'SELLER_METRICS_CLAIMS_VALUE'] *= (60/90)
products.loc[products['SELLER_METRICS_CLAIMS_PERIOD'] == '365 days', 'SELLER_METRICS_CLAIMS_VALUE'] *= (60/365)
products['SELLER_METRICS_CLAIMS_PERIOD'] = '60 days'

# Demoras 
products.loc[products['SELLER_METRICS_DELAY_PERIOD'] == '60 months', 'SELLER_METRICS_DELAY_VALUE'] *= (60/(30*60))
products.loc[products['SELLER_METRICS_DELAY_PERIOD'] == '3 months', 'SELLER_METRICS_DELAY_VALUE'] *= (60/90)
products.loc[products['SELLER_METRICS_DELAY_PERIOD'] == '365 days', 'SELLER_METRICS_DELAY_VALUE'] *= (60/365)
products['SELLER_METRICS_DELAY_PERIOD'] = '60 days'

## Ajustes a variables del shipping/envio

# Cambio de nan to not_specified para SHIPPING_LOGISTIC_TYPE
products.loc[products['SHIPPING_LOGISTIC_TYPE'].isna(), 'SHIPPING_LOGISTIC_TYPE'] = 'not_specified'

In [8]:
# Crea un dataframe con los productos que tienen descuentos
discount_products = products.copy()
discount_products = discount_products.loc[discount_products['ITEM_DISCOUNT_VALUE'] > 0].reset_index(drop=True)
print('LAS DIMENSIONES DE LOS PRODUCTOS CON DESCUENTOS SON: {}'.format(discount_products.shape))

LAS DIMENSIONES DE LOS PRODUCTOS CON DESCUENTOS SON: (988, 53)


## Caracterización de los Descuentos: General

En esta sección se tratan de encontrar las principales métricas, findings e insights de manera general sobre los descuentos.

In [9]:
# Cuantos productos y que porcentaje es del total obtenido (la muestra que se construyo) tienen descuentos
print('EN TOTAL HAY {} PRODUCTOS QUE TIENEN DESCUENTO'.format(discount_products.shape[0]), '\n')
print('LO QUE CORRESPONDE A UN {}% DEL TOTAL DE PRODUCTOS EN EL DATASET'.format(np.round(discount_products.shape[0]/products.shape[0]*100, 2)))

EN TOTAL HAY 988 PRODUCTOS QUE TIENEN DESCUENTO 

LO QUE CORRESPONDE A UN 6.93% DEL TOTAL DE PRODUCTOS EN EL DATASET


Recordemos que teníamos __14256__ productos (después de hacer algunos filtrados generales) y de estos __988__ (el __6.93%__) tenían descuento. Un porcentaje que debería ser muy diferente por categoría. Así que continuemos con el análisis:

In [72]:
# Genera un dataset con el total de items por categoria, el total de items con descuento por categoria
# y el porcentaje de items con descuento por categoria
products_per_category = products.groupby(['SEARCH_CATEGORY_NAME'])[['ITEM_ID']].count().reset_index()
discount_products_per_category = discount_products.groupby(['SEARCH_CATEGORY_NAME'])[['ITEM_ID']].count().reset_index()

percentage_discounts_per_category = pd.merge(products_per_category, discount_products_per_category, on='SEARCH_CATEGORY_NAME',
                                             suffixes=('', '_WITH_DISCOUNT'), how='left').fillna(0)
percentage_discounts_per_category['PORCENTAJE_DE_ITEMS_CON_DESCUENTO'] = percentage_discounts_per_category['ITEM_ID_WITH_DISCOUNT']
percentage_discounts_per_category['PORCENTAJE_DE_ITEMS_CON_DESCUENTO'] /= percentage_discounts_per_category['ITEM_ID']
percentage_discounts_per_category['PORCENTAJE_DE_ITEMS_CON_DESCUENTO'] *= 100 # Llevalo a de 0 a 100
percentage_discounts_per_category.sort_values(by='PORCENTAJE_DE_ITEMS_CON_DESCUENTO', ascending=False, inplace=True)
percentage_discounts_per_category.reset_index(drop=True, inplace=True)
percentage_discounts_per_category

,SEARCH_CATEGORY_NAME,ITEM_ID,ITEM_ID_WITH_DISCOUNT,PORCENTAJE_DE_ITEMS_CON_DESCUENTO
0,Electrodomésticos,461,132.0,28.633406
1,Juegos y Juguetes,460,117.0,25.434783
2,Celulares y Teléfonos,481,114.0,23.700624
3,"Electrónica, Audio y Video",403,82.0,20.347395
4,Deportes y Fitness,472,76.0,16.101695
5,Accesorios para Vehículos,496,53.0,10.685484
6,Computación,479,49.0,10.229645
7,Consolas y Videojuegos,495,42.0,8.484848
8,Herramientas y Construcción,467,38.0,8.137045
9,Ropa y Accesorios,471,31.0,6.581741


In [84]:
# Se guarda el top 10 de categorias en cuanto al porcentaje de productos con descuento
top = 10
top_ten_categorias = percentage_discounts_per_category.head(top).loc[:, 'SEARCH_CATEGORY_NAME'].tolist()
print('CATEGORIAS QUE LIDERAN EN CUANTO AL PORCENTAJE DE ITEMS CON DESCUENTO:', top_ten_categorias)

CATEGORIAS QUE LIDERAN EN CUANTO AL PORCENTAJE DE ITEMS CON DESCUENTO: ['Electrodomésticos', 'Juegos y Juguetes', 'Celulares y Teléfonos', 'Electrónica, Audio y Video', 'Deportes y Fitness', 'Accesorios para Vehículos', 'Computación', 'Consolas y Videojuegos', 'Herramientas y Construcción', 'Ropa y Accesorios']


Es claro que todas las categorías relacionadas a ítems de tecnología hacen parte de las categorías top en cuanto al porcentaje de productos ofertados con descuento, como por ejemplo:

1. Electrodomésticos (28.63%).
2. Celulares y Teléfonos (23.70%).
3. Electrónica, Audio y Video (20.35%).
4. Computación (10.23%).
5. Consolas y Videojuegos (8.48%).

En el top 10 aparecen también categorías como: 

1. Juegos y Juguetes (25.43%). 
2. Deportes y Fitness (16.10%).
3. Accesorios para Vehículos (10.69%).
4. Herramientas y Construcción (8.14%) .
5. Ropa y Accesorios (6.58%).

Pero ¿Será que este top 10 de categorías, ordenadas por el porcentaje de productos por descuentos, son las categorías que lideran la venta total? Veamos.

In [85]:
# Se ordenan las categorias por venta total (cantidad*precio)
# No olvidemos que sold_quantity es una aproximacion y que realmente hace parte de un rango, pero nos da 
# una referencia de cuando vende en dinero cada item
var_ventas = 'ITEM_SELL_TOTAL'
ventas_per_category = products.groupby('SEARCH_CATEGORY_NAME')[[var_ventas]].sum().sort_values(by=var_ventas, ascending=False).reset_index()
ventas_per_category

,SEARCH_CATEGORY_NAME,ITEM_SELL_TOTAL
0,Celulares y Teléfonos,2.358283e+10
1,Computación,2.136634e+10
2,Consolas y Videojuegos,1.776000e+10
3,Electrodomésticos,1.712423e+10
4,Deportes y Fitness,1.707067e+10
5,"Electrónica, Audio y Video",1.394926e+10
6,Hogar y Muebles,1.150002e+10
7,Herramientas y Construcción,9.481478e+09
8,Industrias y Oficinas,8.007053e+09
9,Accesorios para Vehículos,7.851461e+09


Salvo __Industrias y Oficinas__, todos las categorías del top 10 por venta coinciden con las del top 10 por porcentaje de items con descuento, lo cual permite concluir que, de manera general y para las categorías "comunes", a nivel de categoría hay una correlación positiva entre la __venta__ y el __porcentaje de ítems con descuento__. Tratemos de hacer una rápida verificación de nuestrra hipótesis con una gráfica de porcentaje de ítems con descuento vs venta donde cada dato es una categoría:

In [122]:
# Construye dataframe con venta y % descuentos y grafica en eje x % descuentos y en eje y venta
ventas_y_descuentos = pd.merge(ventas_per_category, percentage_discounts_per_category, on='SEARCH_CATEGORY_NAME', how='inner')

fig = px.scatter(ventas_y_descuentos, x="PORCENTAJE_DE_ITEMS_CON_DESCUENTO", y=var_ventas, trendline="ols",
                 title='PORCENTAJE DE ITEMS CON DESCUENTO VS VENTA POR CATEGORIA',
                 hover_data={'SEARCH_CATEGORY_NAME': True}, height=400)
fig.update_layout(titlefont = dict(size=20),
                  xaxis=dict(title="PORCENTAJE DE ITEMS CON DESCUENTO",titlefont=dict(size=18)),
                  yaxis=dict(title="VENTA TOTAL",titlefont=dict(size=18)),
                  plot_bgcolor='rgb(255,255,255)', )

# Imprime el pvalue de la hipotesis del modelo lineal simple
ols_results = px.get_trendline_results(fig).iloc[0,0]
p_value =pd.read_html(ols_results.summary().tables[1].as_html(),header=0,index_col=0)[0].loc['x1', 'P>|t|']
print('EL P-VALOR ES {}'.format(p_value))

fig.show()

EL P-VALOR ES 0.0


Con un p-valor significativo (lo que reforzaría nuestra hipótesis) se puede concluir que hay una correlación positiva entre el porcentaje de ítems con descuento y la venta total en una categoría. Sin embargo, __para mí__, verificar de manera mucho más formal esta tesis requiere de analizar otros factores como la naturaleza, la reputación y el awareness de cada categoría dentro del marketplace, los vendedores y diferentes métricas de estos en cada categoría, entre otros. 

Una categoría que se podría caracterizar como "diferente" es __Juegos y Juguetes__ que se posición como la segunda en cuanto a porcentaje de ítems con descuento pero es la número 15 en cuanto a venta. 

Ahora, entremos al mundo de los productos con descuentos y entendamos de manera general como se distribuyen los descuentos para cada una de las categorías (haciendo un enfasis en las categorías top).

In [114]:
# Genera un dataset con el total de items por categoria, el total de items con descuento por categoria
# y el porcentaje de items con descuento por categoria
discount_products_top = discount_products.loc[discount_products['SEARCH_CATEGORY_NAME'].isin(top_ten_categorias)]
discount_products_per_category_mean = discount_products_top.groupby(['SEARCH_CATEGORY_NAME'])[['ITEM_DISCOUNT_RATE']].mean().reset_index()
discount_products_per_category_median = discount_products_top.groupby(['SEARCH_CATEGORY_NAME'])[['ITEM_DISCOUNT_RATE']].median().reset_index()
discount_products_per_category_std = discount_products_top.groupby(['SEARCH_CATEGORY_NAME'])[['ITEM_DISCOUNT_RATE']].std().reset_index()
discount_products_per_category = pd.merge(discount_products_per_category_mean, discount_products_per_category_std, on='SEARCH_CATEGORY_NAME',
                                          how='inner', suffixes=('_MEAN', '_STD'))
discount_products_per_category = pd.merge(discount_products_per_category, discount_products_per_category_median, on='SEARCH_CATEGORY_NAME',
                                          how='inner').rename(columns={'ITEM_DISCOUNT_RATE': 'ITEM_DISCOUNT_RATE_MEDIAN'})
discount_products_per_category.sort_values(by='ITEM_DISCOUNT_RATE_MEDIAN', ascending=False, inplace=True)
discount_products_per_category.reset_index(drop=True, inplace=True)
sorted_dict = discount_products_per_category['SEARCH_CATEGORY_NAME'].tolist()
sorted_dict = {k: v for v,k in enumerate(sorted_dict)}
discount_products_per_category

,SEARCH_CATEGORY_NAME,ITEM_DISCOUNT_RATE_MEAN,ITEM_DISCOUNT_RATE_STD,ITEM_DISCOUNT_RATE_MEDIAN
0,Ropa y Accesorios,0.408094,0.115562,0.400000
1,"Electrónica, Audio y Video",0.295670,0.131513,0.300083
2,Celulares y Teléfonos,0.279006,0.142023,0.286896
3,Consolas y Videojuegos,0.396167,0.226710,0.286838
4,Deportes y Fitness,0.302534,0.138329,0.270270
5,Accesorios para Vehículos,0.254427,0.122009,0.264341
6,Computación,0.256403,0.119119,0.260052
7,Electrodomésticos,0.250628,0.105798,0.252965
8,Juegos y Juguetes,0.252178,0.091409,0.250000
9,Herramientas y Construcción,0.205726,0.121745,0.195133


In [121]:
discount_products_top_sorted = discount_products_top.sort_values(by=['SEARCH_CATEGORY_NAME'], key=lambda x: x.map(sorted_dict))
fig = px.box(discount_products_top_sorted, x="SEARCH_CATEGORY_NAME", y="ITEM_DISCOUNT_RATE", color="SEARCH_CATEGORY_NAME",)
fig.update_traces(quartilemethod="exclusive")
fig.update_layout(titlefont = dict(size=20),
                  xaxis=dict(title="CATEGORIA",titlefont=dict(size=18)),
                  yaxis=dict(title="DESCUENTO",titlefont=dict(size=18)),
                  plot_bgcolor='rgb(255,255,255)', legend_title='CATEGROIAS')
fig.show()

Se puede ver __Ropa y Accesarios__ lidera, y de manera muy clara, el ranking por descuento mediano (y de hecho también promedio), así como el hecho de que para esta categoría no hay tanta variabilidad como se presentan en las otras (salvo por __Juegos y Jueguetes__). El ranking continua seguido de __Electrónica, Audio y Video__ y luego __Celulares y Teléfonos__ aunque no es claro que en mediana no hay una diferencia en los descuentos para las categorías de la 2da a la 9na. 

Otro caso único es el __Consolas y Videojuegos__ donde la variabilidad es altísima y sobre todo hacia arriba, llega a tener un descuento máximo de más de 70% de descuento.

Por otro lado, es importante resaltar que hay una cantidad de ítems notables fuera del intervalo de cuantiles [0.25, 0.75] para las categorías de __Juegos y Juguetes__ y __Ropa y Accesorios__.

Ahora, vamos a continuar con una análisis más profundo de los descuentos para algunas de las categorías más top (por decisión __personal__ preferí enfocarme en las cinco categorías relacionadas a tecnología): 

1. Electrodomésticos.
2. Celulares y Teléfonos.
3. Electrónica, Audio y Video.
4. Computación.
5. Consolas y Videojuegos.

## Caracterización de los Descuentos: Celulares y Teléfonos

# Fin del Documento